In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.layers import Input,Dense,Flatten,Lambda,Conv2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob


In [3]:
print(tf.__version__)

2.1.0


In [4]:
image_size=[224,224]

train_path=r"C:\Users\Lenovo\Downloads\Dataset\Train"
test_path=r"C:\Users\Lenovo\Downloads\Dataset\Test"

In [5]:
vgg19=VGG19(input_shape=image_size+[3],weights='imagenet',include_top=False)

In [6]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
#dont train the existing weight so, we will use

for layer in vgg19.layers:
    layer.trainable=False

In [8]:
#now flatten the output of vgg19

x=Flatten()(vgg19.output)

In [9]:
print(x)
print(vgg19.input)

Tensor("flatten/Identity:0", shape=(None, 25088), dtype=float32)
Tensor("input_1:0", shape=(None, 224, 224, 3), dtype=float32)


In [10]:
prediction=Dense(2,activation='softmax')(x)
model=Model(inputs=vgg19.input, outputs=prediction)

In [11]:
print(prediction)

Tensor("dense/Identity:0", shape=(None, 2), dtype=float32)


In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
from tensorflow.keras.layers import MaxPooling2D

In [14]:
#model=Sequential()
#model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation='relu',inpu))

In [15]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
#use ImageDataGenerator to import images from folder

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=.2,
                                zoom_range=.2,
                                horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [18]:
training_set=train_datagen.flow_from_directory(r"C:\Users\Lenovo\Downloads\Dataset\Train",
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 416 images belonging to 2 classes.


In [19]:
testing_set=test_datagen.flow_from_directory(r"C:\Users\Lenovo\Downloads\Dataset\Test",
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode='categorical')

Found 134 images belonging to 2 classes.


# fit model

In [20]:
history=model.fit(training_set,
                  validation_data=testing_set,
                  epochs=20,
                  steps_per_epoch=len(training_set),
                  validation_steps=len(testing_set))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 13 steps, validate for 5 steps
Epoch 1/20
13/13 [==============================] - 182s 14s/step - loss: 1.3642 - accuracy: 0.5096 - val_loss: 0.4764 - val_accuracy: 0.7090
Epoch 2/20
13/13 [==============================] - 179s 14s/step - loss: 0.6144 - accuracy: 0.6635 - val_loss: 0.6984 - val_accuracy: 0.5821
Epoch 3/20
13/13 [==============================] - 179s 14s/step - loss: 0.4300 - accuracy: 0.7957 - val_loss: 0.4471 - val_accuracy: 0.7313
Epoch 4/20
13/13 [==============================] - 179s 14s/step - loss: 0.3919 - accuracy: 0.8293 - val_loss: 0.5087 - val_accuracy: 0.7015
Epoch 5/20
13/13 [==============================] - 178s 14s/step - loss: 0.3218 - accuracy: 0.8750 - val_loss: 0.3375 - val_accuracy: 0.8134
Epoch 6/20
13/13 [==============================] - 179s 14s/step - loss: 0.2845 - accuracy: 0.9038 - val_loss: 0.3656 - val_accuracy: 0.8284
Epoch 7/20
13/13 [==============================] - 180s 

In [21]:
from tensorflow.keras.models import load_model

model.save('model_vgg19.h5')

In [144]:
y_pred=model.predict(testing_set)

In [145]:
print(y_pred)

[[9.98176813e-01 1.82325009e-03]
 [9.87141728e-01 1.28582204e-02]
 [1.59373507e-01 8.40626538e-01]
 [6.29627854e-02 9.37037170e-01]
 [9.96059299e-01 3.94067820e-03]
 [4.21305060e-01 5.78694940e-01]
 [6.25144541e-01 3.74855489e-01]
 [8.11493874e-01 1.88506126e-01]
 [9.34343934e-01 6.56560957e-02]
 [9.75928962e-01 2.40710489e-02]
 [9.77187216e-01 2.28127539e-02]
 [9.87430811e-01 1.25692273e-02]
 [2.49166489e-01 7.50833511e-01]
 [9.77825224e-02 9.02217507e-01]
 [6.40782773e-01 3.59217167e-01]
 [7.01026917e-01 2.98973083e-01]
 [9.93860245e-01 6.13974174e-03]
 [6.50423467e-01 3.49576503e-01]
 [7.11006165e-01 2.88993835e-01]
 [9.82574284e-01 1.74257811e-02]
 [9.57245886e-01 4.27541211e-02]
 [2.92039543e-01 7.07960486e-01]
 [7.92492092e-01 2.07507938e-01]
 [1.48008063e-01 8.51991951e-01]
 [8.42340529e-01 1.57659471e-01]
 [1.07640944e-01 8.92359078e-01]
 [9.20334518e-01 7.96654597e-02]
 [9.89754140e-01 1.02458745e-02]
 [9.95863318e-01 4.13661869e-03]
 [9.44961429e-01 5.50385751e-02]
 [3.305733

In [146]:
import numpy as np
y_pred=np.argmax(y_pred,axis=1)
y_pred

array([0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0], dtype=int64)

# check the prediction

In [147]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [148]:
model=load_model('model_vgg19.h5')

In [167]:
img=image.load_img(r"C:\Users\Lenovo\Downloads\cell_images\cell_images\Parasitized\C182P143NThinF_IMG_20151201_172759_cell_13.png",target_size=(224,224))

In [168]:
x=image.img_to_array(img)

In [169]:
x.shape

(224, 224, 3)

In [170]:
x=x/255

# expand the image dimension

In [171]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [172]:
model.predict(img_data)

array([[0.41052452, 0.5894755 ]], dtype=float32)

In [173]:
prediction=np.argmax(model.predict(img_data),axis=1)
prediction

array([1], dtype=int64)

In [174]:
if (prediction==1):
    print("Uninfected")
else:
    print('infected')

Uninfected
